# Web Scraping Assignment 3 : Solutions

#### Q1. Write a python program which searches all the product under a particular product vertical from www.amazon.in. The product verticals to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [1]:
# Importing Libraries
import selenium
import pandas as pd
import time
from bs4 import BeautifulSoup

# Importing selenium webdriver 
from selenium import webdriver

# Importing required Exceptions which needs to handled
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException

#Importing requests
import requests

# importing regex
import re

In [2]:
# Activating the chrome browser
driver = webdriver.Chrome(r"C:\Users\HP\Desktop\Data Trained\Flip Robo\Assignment 3\WEB-SCRAPING-ASSIGNMENT-3-Solutions (1).ipynb")

In [3]:
# Opening the homepage of Amazon.in
driver.get('https://www.amazon.in/')
# Asking the user to input the keywords he/she wants to search
user_inp = input('Enter the product you want to search : ')

Enter the product you want to search : guitar


In [4]:
search_bar = driver.find_element_by_id("twotabsearchtextbox")    # Locating searc_bar by id
search_bar.clear()                                               # clearing search_bar
search_bar.send_keys(user_inp)                                   # sending user input to search bar
search_button = driver.find_element_by_xpath('//div[@class="nav-search-submit nav-sprite"]/span/input')       # Locating search_button by xpath
search_button.click()                                                                # Clicking the button to start search

#### Q2. 2.	In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a dataframe and csv. In case if any product vertical has less than 3 pages in search results then scrape all the products available under that product vertical. Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of Ratings", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [5]:
start_page = 0
end_page = 3
urls = []
for page in range(start_page,end_page+1):
    try:
        page_urls = driver.find_elements_by_xpath('//a[@class="a-link-normal s-no-outline"]')
        
        # appending all the urls on current page to urls list
        for url in page_urls:
            url = url.get_attribute('href')     # Scraping the url from webelement
            if url[0:4]=='http':                # Checking if the scraped data is a valid url or not
                urls.append(url)                # Appending the url to urls list
        print("Product urls of page {} has been scraped.".format(page+1))
        
        # Moving to next page
        nxt_button = driver.find_element_by_xpath('//li[@class="a-last"]/a')      # Locating the next_button which is active
        if nxt_button.text == 'Next→':                                            # Checking if the button located is next button
            nxt_button.click()                                                    # Clicking the next button
            time.sleep(5)                                                         # time delay of 5 seconds
        # If the current active button is not next button, the we will check if the next button is inactive or not    
        elif driver.find_element_by_xpath('//li[@class="a-disabled a-last"]/a').text == 'Next→':    
            print("No new pages exist. Breaking the loop")  # Printing message and breakinf loop if we have reached the last page
            break
            
    except StaleElementReferenceException as e:             # Handling StaleElement Exception   
        print("Stale Exception")
        next_page = nxt_button.get_attribute('href')        # Extracting the url of next page
        driver.get(next_page)                               # ReLoading the next page

Product urls of page 1 has been scraped.
Product urls of page 2 has been scraped.
Product urls of page 3 has been scraped.
Product urls of page 4 has been scraped.


In [6]:
prod_dict = {}
prod_dict['Brand']=[]
prod_dict['Name']=[]
prod_dict['Rating']=[]
prod_dict['No. of ratings']=[]
prod_dict['Price']=[]
prod_dict['Return/Exchange']=[]
prod_dict['Expected Delivery']=[] 
prod_dict['Availability']=[]
prod_dict['Other Details']=[]
prod_dict['URL']=[]

In [7]:
for url in urls[:4]:
    driver.get(url)                                                        # Loading the webpage by url
    print("Scraping URL = ", url)
    #time.sleep(2)
    
    try:
        brand = driver.find_element_by_xpath('//a[@id="bylineInfo"]')      # Extracting Brand from xpath
        prod_dict['Brand'].append(brand.text)
    except NoSuchElementException:
        prod_dict['Brand'].append('-')
    
    try:
        name = driver.find_element_by_xpath('//h1[@id="title"]/span')      # Extracting Name from xpath
        prod_dict['Name'].append(name.text)
    except NoSuchElementException:
        prod_dict['Name'].append('-')
    
    try:
        rating = driver.find_element_by_xpath('//span[@id="acrPopover"]')  # Extracting Ratings from xpath
        prod_dict['Rating'].append(rating.get_attribute("title"))
    except NoSuchElementException:
        prod_dict['Rating'].append('-')
    
    try:
        n_rating = driver.find_element_by_xpath('//a[@id="acrCustomerReviewLink"]/span')     # Extracting no. of Ratings from xpath
        prod_dict['No. of ratings'].append(n_rating.text)
    except NoSuchElementException:
        prod_dict['No. of ratings'].append('-')
    
    try:
        price = driver.find_element_by_xpath('//span[@id="priceblock_ourprice"]')            # Extracting Price from xpath
        prod_dict['Price'].append(price.text)
    except NoSuchElementException:
        prod_dict['Price'].append('-')
    try:                                                                                     # Extracting Return/Exchange policy from xpath
        ret = driver.find_element_by_xpath('//div[@data-name="RETURNS_POLICY"]/span/div[2]/a')
        prod_dict['Return/Exchange'].append(ret.text)
    except NoSuchElementException:
        prod_dict['Return/Exchange'].append('-')
    try:
        delivry = driver.find_element_by_xpath('//div[@id="ddmDeliveryMessage"]/b')         # Extracting Expected Delivery from xpath
        prod_dict['Expected Delivery'].append(delivry.text)
    except NoSuchElementException:
        prod_dict['Expected Delivery'].append('-')
    
    try:
        avl = driver.find_element_by_xpath('//div[@id="availability"]/span')                # Extracting Availability from xpath
        prod_dict['Availability'].append(avl.text)
    except NoSuchElementException:
        prod_dict['Availability'].append('-')
    
    try:                                                                                    # Extracting Other Details from xpath
        dtls = driver.find_element_by_xpath('//ul[@class="a-unordered-list a-vertical a-spacing-mini"]')
        prod_dict['Other Details'].append('  ||  '.join(dtls.text.split('\n')))
    except NoSuchElementException:
        prod_dict['Other Details'].append('-')
    
    prod_dict['URL'].append(url)                                                            # Saving url
    time.sleep(2)

Scraping URL =  https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A05211881ZGJIHX360ZMV&url=%2FKadence-Frontier-Acoustic-Guitar-Equalizer%2Fdp%2FB01HRIHJWY%2Fref%3Dsr_1_1_sspa%3Fdchild%3D1%26keywords%3Dguitar%26qid%3D1629740541%26sr%3D8-1-spons%26psc%3D1&qualifier=1629740541&id=2621138676573789&widgetName=sp_atf
Scraping URL =  https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A0928654VYD9HPIWAKBZ&url=%2FKadence-Frontier-Natural-Acoustic-Foldable%2Fdp%2FB01CJJGJ04%2Fref%3Dsr_1_2_sspa%3Fdchild%3D1%26keywords%3Dguitar%26qid%3D1629740541%26sr%3D8-2-spons%26psc%3D1&qualifier=1629740541&id=2621138676573789&widgetName=sp_atf
Scraping URL =  https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A087933837LQ4BBEWM3H5&url=%2FGUITAR-BRO-Acoustic-Beginner-Accessories%2Fdp%2FB07T84GC8L%2Fref%3Dsr_1_3_sspa%3Fdchild%3D1%26keywords%3Dguitar%26qid%3D1629740541%26sr%3

In [8]:
prod_df = pd.DataFrame.from_dict(prod_dict)
prod_df

Brand  \
0     Visit the Kadence Store   
1     Visit the Kadence Store   
2  Visit the Guitar Bro Store   
3             Brand: Honestum   

                                                Name              Rating  \
0  Kadence Frontier Jumbo Semi Acoustic Guitar Wi...  3.9 out of 5 stars   
1  Kadence Frontier Series, Natural Acoustic Guit...  3.8 out of 5 stars   
2  GUITAR BRO - COMBO (Black Acoustic Guitar for ...  3.7 out of 5 stars   
3  Honestum 4-string acoustic guitar learning kid...                   -   

  No. of ratings      Price     Return/Exchange Expected Delivery  \
0    984 ratings          -  7 Days Replacement  Thursday, Aug 26   
1     75 ratings          -  7 Days Replacement  Thursday, Aug 26   
2     28 ratings  ₹6,499.00  7 Days Replacement    Aug 28 - Sep 1   
3              -    ₹999.00  7 Days Replacement  Saturday, Aug 28   

  Availability                                      Other Details  \
0    In stock.  QUALITY STRINGS ---The surface is coated with ...   
1    In stock.  Product Type : Acoustic Guitar  ||  Brand:Kade...   
2    In stock.  GUITAR BRO +20 mins FREE VIDEO Demo - is a lea...   
3    In stock.  Very Crisp and Clear Sound of Strings  ||  Thi...   

                                                 URL  
0  https://www.amazon.in/gp/slredirect/picassoRed...  
1  https://www.amazon.in/gp/slredirect/picassoRed...  
2  https://www.amazon.in/gp/slredirect/picassoRed...  
3  https://www.amazon.in/gp/slredirect/picassoRed...

In [9]:
#saving data to csv
prod_df.to_csv('Amazon_{}.csv'.format(user_inp))

#### Q3. Write a python program to access the search bar and search button on images.google.com and scrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’.

In [10]:
# Opening images.google.com
driver.get('https://images.google.com/')

In [11]:
driver.get('https://images.google.com/')
search_bar = driver.find_element_by_xpath('//*[@id="sbtc"]/div/div[2]/input')    # Finding the search bar using it's xpath
search_bar.send_keys("fruits")       # Inputing "banana" keyword to search rock images
search_button = driver.find_element_by_xpath('//*[@id="sbtc"]/button')    # Finding the xpath of search button
search_button.click()        # Clicking the search button
print("start scrolling to generate more images on the page...")
# 500 time we scroll down by 10000 in order to generate more images on the website
for _ in range(500):
    driver.execute_script("window.scrollBy(0,10000)")
    

images = driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')
img_urls = []
img_data = []
for image in images:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)
len(img_urls)
for i in range(len(img_urls)):
    if i >= 100:
        break
    print("Downloading {0} of {1} images" .format(i, 100))
    response= requests.get(img_urls[i])
    file = open(r"C:\Users\HP\Desktop\Data Trained\Flip Robo\Assignment 3\fruits")
    file.write(response.content)


start scrolling to generate more images on the page...


#### Q4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [15]:
# Asking for user input
prod = input(" Enter the name of the mobile phone you want to search : ")
driver.get('https://www.flipkart.com/')
time.sleep(3)
try:
    login_X_button = driver.find_element_by_xpath('//button[@class="_2KpZ6l _2doB4z"]')                      # Button to close login popup
    login_X_button.click()
except NoSuchElementException : 
    print("No Login page")
search_bar = driver.find_element_by_xpath('//*[@id="container"]/div/div[1]/div[1]/div[2]/div[2]/form/div/div/input')    # Finding the search bar using it's xpath
search_bar.clear()               # Clearing the search bar
search_bar.send_keys(prod)       # Inputing keyword to search
search_button = driver.find_element_by_xpath('//button[@class="L0Z3Pu"]')    # Finding the xpath of search button
search_button.click()        # Clicking the search button

 Enter the name of the mobile phone you want to search : Oneplus Nord


In [20]:
# Fetching urls of phones coming on 1st page
flip_urls = []
urls = driver.find_elements_by_xpath('//a[@class="s1Q9rs"]')
for url in urls:
    flip_urls.append(url.get_attribute("href"))

In [21]:
len(flip_urls)

40

In [22]:
flip_dict = {}
flip_dict["Brand"] = []
flip_dict["Smartphone"] = []
flip_dict["Colour"] = []
flip_dict["RAM"] = []
flip_dict["Storage(ROM)"] = []
flip_dict["Primary Camera"] = []
flip_dict["Secondary Camera"] = []
flip_dict["Display Size"] = []
flip_dict["Display Resolution"] = []
flip_dict["Processor"] = []
flip_dict["Processor Cores"] = []
flip_dict["Battery Capacity"] = []
flip_dict["Battery Type"] = []
flip_dict["Price"] = []
flip_dict["URL"] = []

In [23]:
# Scraping data from each url
for url in flip_urls:
    driver.get(url)    # Saving url                                                     
    print("Scraping URL = ", url)
    flip_dict['URL'].append(url)                                                          # Loading the webpage by url
    time.sleep(2)
    
    try:
        read_more = driver.find_element_by_xpath('//button[@class="_2KpZ6l _1FH0tX"]')     # Button for expanding the specs
        read_more.click()
    except NoSuchElementException:
        print("Exception Occured. Moving to next page")
    
    try:
        brand = driver.find_element_by_xpath('//span[@class="B_NuCI"]')      # Extracting Brand from xpath
        flip_dict["Brand"].append(brand.text.split()[0])
    except NoSuchElementException:
        flip_dict['Brand'].append('-')
        
    try:
        price = driver.find_element_by_xpath('//div[@class="_30jeq3 _16Jk6d"]')      # Extracting Brand from xpath
        flip_dict['Price'].append(price.text)
    except NoSuchElementException:
        flip_dict['Price'].append('-')
        
    try:
        name = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][1]/table/tbody/tr[3]/td[2]/ul/li')      # Extracting Brand from xpath
        flip_dict['Smartphone'].append(name.text)
    except NoSuchElementException:
        flip_dict['Smartphone'].append('-')
    
    try:
        color = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][1]/table/tbody/tr[4]/td[2]/ul/li')      # Extracting Name from xpath
        flip_dict['Colour'].append(color.text)
    except NoSuchElementException:
        flip_dict['Colour'].append('-')
    
    try:
        disp_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][2]/div')
        if disp_chk.text != "Display Features" : raise NoSuchElementException
        disp_size = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][2]/table[1]/tbody/tr[1]/td[2]/ul/li')  # Extracting Ratings from xpath
        flip_dict['Display Size'].append(disp_size.text)
    except NoSuchElementException:
        flip_dict['Display Size'].append('-')
    
    try:
        disp_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][2]/div')
        if disp_chk.text != "Display Features" : raise NoSuchElementException
        disp_res = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][2]/table[1]/tbody/tr[2]/td[2]/ul/li')     # Extracting no. of Ratings from xpath
        flip_dict['Display Resolution'].append(disp_res.text)
    except NoSuchElementException:
        flip_dict['Display Resolution'].append('-')
    
    try:
        pro_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[2]/td[1]')
        if pro_chk.text != "Processor Type" : raise NoSuchElementException
        processor = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[2]/td[2]/ul/li')   # Extracting Price from xpath
        flip_dict['Processor'].append(processor.text)
    except NoSuchElementException:
        flip_dict['Processor'].append('-')
    
    try:                                                                                     # Extracting Return/Exchange policy from xpath
        core_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[3]/td[1]')
        if core_chk.text != "Processor Core" :
            core_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[2]/td[1]')
            if core_chk.text != "Processor Core" : 
                raise NoSuchElementException
            else :
                cores = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[2]/td[2]/ul/li')
        else :
            cores = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[3]/td[2]/ul/li')
        flip_dict['Processor Cores'].append(cores.text)
    except NoSuchElementException:
        flip_dict['Processor Cores'].append('-')
    
    try:
        rom = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[1]/td[2]/ul/li')         # Extracting Expected Delivery from xpath
        flip_dict['Storage(ROM)'].append(rom.text)
    except NoSuchElementException:
        flip_dict['Storage(ROM)'].append('-')
    
    try:
        ram = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[2]/td[2]/ul/li')                # Extracting Availability from xpath
        flip_dict['RAM'].append(ram.text)
    except NoSuchElementException:
        flip_dict['RAM'].append('-')
    
    try:                                                                                    # Extracting Other Details from xpath
        pri_cam = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[2]/td[2]/ul/li')
        flip_dict['Primary Camera'].append(pri_cam.text)
    except NoSuchElementException:
        flip_dict['Primary Camera'].append('-')
    
    try:                                                                                    # Extracting Other Details from xpath
        cam_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[1]')
        if cam_chk != "Secondary Camera" : 
            if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[1]').text == "Secondary Camera":
                sec_cam = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[2]/ul/li')
            else :
                raise NoSuchElementException
        else :
            sec_cam = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[2]/ul/li')
        flip_dict['Secondary Camera'].append(sec_cam.text)
    except NoSuchElementException:
        flip_dict['Secondary Camera'].append('-')
        
    try:
        if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/div').text != "Battery & Power Features" :
            if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/div').text == "Battery & Power Features" :
                bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[1]')
                if bat_chk.text != "Battery Capacity" : raise NoSuchElementException
                bat_cap = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[2]/ul/li')                # Extracting Availability from xpath
            elif driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/div').text == "Battery & Power Features" :
                bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[1]')
                if bat_chk.text != "Battery Capacity" : raise NoSuchElementException
                bat_cap = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[2]/ul/li')
            else:
                raise NoSuchElementException
        else :
            bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[1]')
            if bat_chk.text != "Battery Capacity" : raise NoSuchElementException
            bat_cap = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[2]/ul/li')                # Extracting Availability from xpath
        flip_dict['Battery Capacity'].append(bat_cap.text)
    except NoSuchElementException:
        flip_dict['Battery Capacity'].append('-')
    
    try:
        if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/div').text != "Battery & Power Features" :
            if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/div').text == "Battery & Power Features" :
                bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/table/tbody/tr[2]/td[1]')
                if bat_chk.text != "Battery Type" : raise NoSuchElementException
                bat_typ = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/table/tbody/tr[2]/td[2]/ul/li')
            elif driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/div').text == "Battery & Power Features" :
                bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/table/tbody/tr[2]/td[1]')
                if bat_chk.text != "Battery Type" : raise NoSuchElementException
                bat_typ = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/table/tbody/tr[2]/td[2]/ul/li')
            else:
                raise NoSuchElementException
        else :
            bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr[2]/td[1]')
            if bat_chk.text != "Battery Type" : raise NoSuchElementException
            bat_typ = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr[2]/td[2]/ul/li')                # Extracting Availability from xpath
        flip_dict['Battery Type'].append(bat_typ.text)
    except NoSuchElementException:
        flip_dict['Battery Type'].append('-')
    
                                                               

Scraping URL =  https://www.flipkart.com/mobile-mart-back-cover-oneplus-nord/p/itm4cea46317a8c1?pid=ACCFV3GGCRCT26CH&lid=LSTACCFV3GGCRCT26CHNOE6RC&marketplace=FLIPKART&q=Oneplus+Nord&store=tyy&srno=s_1_1&otracker=search&otracker1=search&fm=SEARCH&iid=27741ae1-aa10-460c-99d0-6313476ce043.ACCFV3GGCRCT26CH.SEARCH&ppt=hp&ppn=homepage&ssid=ullptspimo0000001629741062397&qH=53863d4d978d5074
Scraping URL =  https://www.flipkart.com/shinestar-back-cover-oneplus-nord/p/itm31722dac81fc2?pid=ACCFXPQHRZG67GKY&lid=LSTACCFXPQHRZG67GKYSDKM97&marketplace=FLIPKART&q=Oneplus+Nord&store=tyy&srno=s_1_2&otracker=search&otracker1=search&fm=SEARCH&iid=27741ae1-aa10-460c-99d0-6313476ce043.ACCFXPQHRZG67GKY.SEARCH&ppt=hp&ppn=homepage&ssid=ullptspimo0000001629741062397&qH=53863d4d978d5074
Scraping URL =  https://www.flipkart.com/flipkart-smartbuy-back-cover-oneplus-nord/p/itm421ee85693659?pid=ACCFVGZHHDURYQJ4&lid=LSTACCFVGZHHDURYQJ4RAZRF4&marketplace=FLIPKART&q=Oneplus+Nord&store=tyy&srno=s_1_3&otracker=search&ot

Scraping URL =  https://www.flipkart.com/newselect-back-cover-oneplus-nord/p/itmd32a7b7e923f7?pid=ACCFUHZMS9F7362B&lid=LSTACCFUHZMS9F7362BVTSYSP&marketplace=FLIPKART&q=Oneplus+Nord&store=tyy&srno=s_1_21&otracker=search&otracker1=search&fm=SEARCH&iid=27741ae1-aa10-460c-99d0-6313476ce043.ACCFUHZMS9F7362B.SEARCH&ppt=hp&ppn=homepage&ssid=ullptspimo0000001629741062397&qH=53863d4d978d5074
Scraping URL =  https://www.flipkart.com/karwan-back-cover-oneplus-nord/p/itmdff65fa5f00e7?pid=ACCFV26KZGBA4ZQE&lid=LSTACCFV26KZGBA4ZQEMHXSVF&marketplace=FLIPKART&q=Oneplus+Nord&store=tyy&srno=s_1_22&otracker=search&otracker1=search&fm=SEARCH&iid=27741ae1-aa10-460c-99d0-6313476ce043.ACCFV26KZGBA4ZQE.SEARCH&ppt=hp&ppn=homepage&ssid=ullptspimo0000001629741062397&qH=53863d4d978d5074
Scraping URL =  https://www.flipkart.com/bozti-back-cover-oneplus-nord/p/itm02fa7d1088ca9?pid=ACCFV3TQCFFCB6CH&lid=LSTACCFV3TQCFFCB6CHGT3EJD&marketplace=FLIPKART&q=Oneplus+Nord&store=tyy&srno=s_1_23&otracker=search&otracker1=search

Exception Occured. Moving to next page


In [24]:
print(len(flip_dict["Brand"]), len(flip_dict["Smartphone"]), len(flip_dict["Processor"]), len(flip_dict["Price"]), len(flip_dict['URL']))

40 40 40 40 40


In [25]:
df = pd.DataFrame.from_dict(flip_dict)
df

Brand                                      Smartphone  \
0           Mobile                                    OnePlus Nord   
1       SHINESTAR.                                    Oneplus Nord   
2         Flipkart                                    OnePlus Nord   
3           Mobile                                    OnePlus Nord   
4           DSCASE                                    OnePlus Nord   
5        Wellpoint                OnePlus Nord, Plain, Case, Cover   
6               My                                    Oneplus Nord   
7           MECase                                    OnePlus Nord   
8          COSHIBA                                           Front   
9       SOLIVAGANT                                    Front & Back   
10           FIRST                                    Front & Back   
11            TODO                                           Front   
12            from                                           Front   
13       BRIGHTRON                                               1   
14        A3sprime                                               -   
15            ZYNK                                    OnePlus Nord   
16         GadgetM                                    OnePlus Nord   
17         GadgetM                                    OnePlus Nord   
18         GadgetM                                    OnePlus Nord   
19       COBIERTAS                                    Oneplus Nord   
20       NewSelect                                    OnePlus Nord   
21          KARWAN                                    OnePlus Nord   
22           BOZTI                                    OnePlus Nord   
23       Firstkart                                    Front & Back   
24         Adoniss                                           Front   
25       Firstkart                                    Front & Back   
26               I                                            Back   
27      SOLIVAGANT                                    Front & Back   
28  FlipSmartGuard                                               1   
29           Furst  10 Days Warranty Against Manufacturing Defects   
30           BOZTI                                    OnePlus Nord   
31            ZYNK                                    OnePlus Nord   
32          MECase                                    Oneplus Nord   
33         Kapaver                                    OnePlus Nord   
34              My                                    Oneplus Nord   
35          Micvir                                    OnePlus Nord   
36          KARWAN                                    OnePlus Nord   
37         SAPCASE                                    OnePlus Nord   
38       Firstkart                                    Front & Back   
39        CZARTECH                                           Front   

                                               Colour RAM Storage(ROM)  \
0                                               Black   -            -   
1                                               Black   -            -   
2                                         Transparent   -            -   
3                                                Blue   -            -   
4                                         Transparent   -            -   
5                                                Blue   -            -   
6                                          Multicolor   -            -   
7                                                Blue   -            -   
8                                        ONEPLUS NORD   -            -   
9                                                   2   -            -   
10                                                  2   -            -   
11  First clean your Device with wet-wipe & dry-wi...   -            -   
12                                                  2   -            -   
13                                                Yes   -            -   
14                              

#### Q5. 5.	Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [26]:
# opening google maps
driver.get("https://www.google.co.in/maps")
time.sleep(3)

city = input('Enter City Name : ')                                         # Enter city to be searched
search = driver.find_element_by_id("searchboxinput")                       # locating search bar
search.clear()                                                             # clearing search bar
time.sleep(2)
search.send_keys(city)                                                     # entering values in search bar
button = driver.find_element_by_id("searchbox-searchbutton")               # locating search button
button.click()                                                             # clicking search button
time.sleep(3)

try:
    url_string = driver.current_url
    print("URL Extracted: ", url_string)
    lat_lng = re.findall(r'@(.*)data',url_string)
    if len(lat_lng):
        lat_lng_list = lat_lng[0].split(",")
        if len(lat_lng_list)>=2:
            lat = lat_lng_list[0]
            lng = lat_lng_list[1]
        print("Latitude = {}, Longitude = {}".format(lat, lng))

except Exception as e:
        print("Error: ", str(e))

Enter City Name : Bengaluru
URL Extracted:  https://www.google.co.in/maps/place/Bengaluru,+Karnataka/@13.0023424,77.594624,12z/data=!4m5!3m4!1s0x3bae1670c9b44e6d:0xf8dfc3e8517e4fe0!8m2!3d12.9715987!4d77.5945627
Latitude = 13.0023424, Longitude = 77.594624


#### Q6.	Write a program to scrap details of all the funding deals for second quarter (i.e. July 20 – September 20) from trak.in.

In [27]:
driver.get('https://trak.in/')

In [28]:
button = driver.find_element_by_xpath('//li[@id="menu-item-51510"]/a').get_attribute('href')
driver.get(button)

In [29]:
fund_dict = {}
fund_dict['Date'] = []
fund_dict['Startup Name'] = []
fund_dict['Industry/Vertical'] = []
fund_dict['Sub-Vertical'] = []
fund_dict['Location'] = []
fund_dict['Investor'] = []
fund_dict['Investment Type'] = []
fund_dict['Amount(in USD)'] = []

In [30]:
for i in range(48,51):
    driver.find_element_by_xpath('//div[@id="tablepress-{}_wrapper"]/div/label/select/option[4]'.format(i)).click()

    # Date
    dt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[2]'.format(i))
    for d in dt:
        fund_dict['Date'].append(d.text)

    # Startup Name
    sn = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[3]'.format(i))
    for n in sn:
        fund_dict['Startup Name'].append(n.text)
    
    # Industry/Vertical
    ind = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[4]'.format(i))
    for n in ind:
        fund_dict['Industry/Vertical'].append(n.text)
    
    # Sub-Vertical
    sv = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[5]'.format(i))
    for s in sv:
        fund_dict['Sub-Vertical'].append(s.text)

    # Location
    loc = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[6]'.format(i))
    for l in loc:
        fund_dict['Location'].append(l.text)
    
    # Investor
    inv = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[7]'.format(i))
    for n in inv:
        fund_dict['Investor'].append(n.text)
    
    # Investment Type
    invt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[8]'.format(i))
    for n in invt:
        fund_dict['Investment Type'].append(n.text)
    
    # Amount
    amt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[9]'.format(i))
    for a in amt:
        fund_dict['Amount(in USD)'].append(a.text)
    
fund_df = pd.DataFrame(fund_dict)
fund_df

Date               Startup Name  \
0   15/07/2020                   Flipkart   
1   16/07/2020                    Vedantu   
2   16/07/2020                       Crio   
3   14/07/2020                    goDutch   
4   13/07/2020                   Mystifly   
5   09/07/2020               JetSynthesys   
6   10/07/2020                   gigIndia   
7   15/07/2020                  PumPumPum   
8   14/07/2020                       FLYX   
9   13/07/2020  Open Appliances Pvt. Ltd.   
10  15/08/2020                     Practo   
11  13/08/2020                    Medlife   
12  13/08/2020                  HungerBox   
13  04/08/2020                      Dunzo   
14  11/08/2020                   Terra.do   
15  12/08/2020                  Classplus   
16  14/08/2020                       Niyo   
17  10/08/2020                  ZestMoney   
18  07/08/2020                FreshToHome   
19  13/08/2020                    Eduvanz   
20  03/08/2020                 CrowdPouch   
21  04/08/2020                 DrinkPrime   
22  08/09/2020                     Byju’s   
23  12/09/2020                  mCaffeine   
24  09/09/2020                     Qshala   
25  02/09/2020                      Winzo   
26  09/09/2020                Hippo Video   
27  07/09/2020                    Melorra   
28  07/09/2020                        1mg   
29  31/08/2020                      mfine   
30  31/08/2020                       Apna   
31  03/09/2020                    Railofy   
32  08/09/2020            Cell Propulsion   

                         Industry/Vertical  \
0                               E-commerce   
1                                  EduTech   
2                                  EduTech   
3                                  FinTech   
4                      Airfare Marketplace   
5                 Gaming and Entertainment   
6                              Marketplace   
7                        Automotive Rental   
8                               OTT Player   
9                   Information Technology   
10                              HealthTech   
11                              E-commerce   
12                                FoodTech   
13                   Hyper-local Logistics   
14                                 EduTech   
15                                 EduTech   
16                                 FinTech   
17                                 FinTech   
18                              E-commerce   
19                                 FinTech   
20                                 FinTech   
21                      Water Purification   
22                                 EduTech   
23                           Personal Care   
24                                 EduTech   
25                           Online Gaming   
26  Video Customer Experience(CX) Platform   
27                              E-commerce   
28                              E-commerce   
29                              HealthTech   
30                         Human Resources   
31                          Transportation   
32                              Automobile   

                                         Sub-Vertical  \
0                                          E-commerce   
1                                     Online Tutoring   
2                    Learning Platform for Developers   
3                                      Group Payments   
4   Ticketing, Airline Retailing, and Post-Ticketi...   
5                            Gaming and Entertainment   
6                           Crowd Sourcing, Freelance   
7                           Used Car-leasing platform   
8                            Streaming Social Network   
9               Internet-of-Things Security Solutions   
10                           Health care and Wellness   
11                                    Online Pharmacy   
12                       Online Food Delivery Service   
13                           Online Delivery Services   
14                  Online Climate School, E-learning   
15

In [31]:
fund_df.to_csv("Indian Startups_Q2_2020.csv")

#### Q7. Write a program to scrap all the available details of top 10 gaming laptops from digit.in.

In [32]:
driver.get('https://www.digit.in/')

In [34]:
#clickng on top 10 option 
top_10=driver.find_element_by_xpath("//div[@class='menu']/ul/li[3]/a")
top_10.click()

#clicking on laptops option
laptops=driver.find_element_by_xpath("//div[@class='col-xl']/div[3]/ul/li[3]/a")
laptops.click()

#best gaming laptops link
best_gaming=driver.find_element_by_xpath("//ul[@id='list-unstyled sidebar-list']/a[3]")
driver.get(best_gaming.get_attribute('href'))

#intialising lists
name = []
price = []
OS = []
display = []
processor = []
HDD = []
RAM = []
weight = []
dimension = []
GPU = []

#names
names=driver.find_elements_by_xpath("//div[@class='right-container']/div/a/h3")
for i in names:
    name.append(i.text)
    
#os
os=driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[1]/div/div")
for i in os:
    OS.append(i.text)
    
#display
displays=driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[2]/div/div")
for i in displays:
    display.append(i.text)
    
#processor
processors=driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[3]/div/div")
for i in processors:
    processor.append(i.text)
processor

#memory
memories=driver.find_elements_by_xpath("//div[@class='Spcs-details'][1]/table/tbody/tr[6]/td[1]")#list of specificaion name
memories_spec=driver.find_elements_by_xpath("//div[@class='Spcs-details'][1]/table/tbody/tr[6]/td[3]")#values of specifiations 
for i in range(len(memories)):
        if memories[i].text=='Memory':
            HDD.append(memories_spec[i].text.split('/')[0])
            RAM.append(memories_spec[i].text.split('/')[1])
        else:
            HDD.append('No details available')#append no details as value for memory is missing in some of the laptops
            RAM.append('No details available')#append no details as value for memory is missing in some of the laptops

#weight
weights=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[1]")#list of specificaion name
weight_spec=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[3]")#values of specifiations
for i in range(len(weights)):
        if weights[i].text=='Weight':
            weight.append(weight_spec[i].text)
        
#dimension
dimension=[]
dims=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[1]")#list of specificaion name
dims_spec=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[3]")#values of specifiations
for i in range(len(dims)):
        if dims[i].text=='Dimension':
            dimension.append(dims_spec[i].text)

#graphical processor
GPUs=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[1]")#list of specificaion name
GPUs_spec=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[3]")#values of specifiations
for i in range(len(GPUs)):
        if GPUs[i].text=='Graphics Processor':
            GPU.append(GPUs_spec[i].text)
        
full_specs=[]
urls=driver.find_elements_by_xpath("//div[@class='full-specs']/span")#getting the url of full specs links
for i in urls:
    if i.get_attribute('data-href'):
        full_specs.append(i.get_attribute('data-href'))
    
for i in full_specs:#iterating throug every laptops full specs' page
    driver.get(i)
    try:
        prices=driver.find_element_by_xpath("//div[@class='Block-price']/b")
        price.append(prices.text)
    except NoSuchElementException:#exception handling for no price details
        price.append("No details available")
        
df=pd.DataFrame({"Name":name,
                "Price":price,
                "OS":OS,
                "Display":display,
                "HDD":HDD,
                 "RAM":RAM,
                "processor":processor,
                "weight":weight,
                "Dimension":dimension,
                "Graphical processor":GPU})
df

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@id='laptops']/a[3]"}
  (Session info: chrome=92.0.4515.159)


In [ ]:
df.to_csv("Gaming laptops_digit.csv")